# Stage 3: Morphogen-Regulon Network Analysis

This notebook creates networks connecting morphogens, timing, and medium conditions to regulon activities using GRNBoost2.

**Input**: Consensus regulons from Stage 2, original AUCell matrices
**Output**: Morphogen-regulon interaction networks
**Method**: GRNBoost2 network inference

In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
import warnings
warnings.filterwarnings('ignore')

# Import GRNBoost analysis utilities
import sys
sys.path.append('../src')
from grnboost_analysis import (
    load_morphogen_data,
    run_grnboost_analysis,
    analyze_networks,
    save_network_results
)

print("📊 Stage 3: Morphogen-Regulon Network Analysis")
print("Using GRNBoost2 to connect morphogens to regulon activities")

## 3.1 Load Data

Load the morphogen data and AUCell matrices for network inference.

In [ ]:
# Load morphogen and timing data
print("Loading morphogen data...")
morphogen_data = load_morphogen_data()

print(f"Morphogen features: {list(morphogen_data.columns)}")
print(f"Data shape: {morphogen_data.shape}")
print(f"Sample of morphogen data:")
print(morphogen_data.head())

In [ ]:
# Load AUCell matrices for each cell line
# These should be the results from pySCENIC runs

aucell_data = {}
for cell_line in ['H1', 'WTC', 'H9', 'WIBJ2']:
    
    # Look for AUCell files in the results
    aucell_dir = f"../01_pyscenic_runs/results/{cell_line}"
    
    if os.path.exists(aucell_dir):
        aucell_files = [f for f in os.listdir(aucell_dir) if 'aucell' in f.lower() and f.endswith('.csv')]
        
        if aucell_files:
            # Load the first AUCell file found
            aucell_file = f"{aucell_dir}/{aucell_files[0]}"
            print(f"Loading AUCell data for {cell_line}: {aucell_files[0]}")
            
            aucell_df = pd.read_csv(aucell_file, index_col=0)
            aucell_data[cell_line] = aucell_df
            
            print(f"  Shape: {aucell_df.shape}")
            print(f"  Regulons: {aucell_df.shape[1]}")
        else:
            print(f"❌ No AUCell files found for {cell_line}")
    else:
        print(f"❌ Directory not found: {aucell_dir}")

print(f"\nLoaded AUCell data for {len(aucell_data)} cell lines")

## 3.2 Run GRNBoost Analysis

Use GRNBoost2 to infer networks connecting morphogens to regulon activities.

In [ ]:
# Run GRNBoost analysis for each cell line
network_results = {}

for cell_line, aucell_df in aucell_data.items():
    print(f"\n{'='*50}")
    print(f"Running GRNBoost analysis for {cell_line}")
    print(f"{'='*50}")
    
    try:
        # Run network inference
        network = run_grnboost_analysis(
            morphogen_data=morphogen_data,
            aucell_data=aucell_df,
            cell_line=cell_line
        )
        
        network_results[cell_line] = network
        print(f"✅ Completed network inference for {cell_line}")
        print(f"   Network edges: {len(network)}")
        
    except Exception as e:
        print(f"❌ Error in {cell_line}: {str(e)}")
        continue

print(f"\n🎉 Network inference completed for {len(network_results)} cell lines")

## 3.3 Analyze Networks

Analyze the inferred networks to identify significant morphogen-regulon interactions.

In [ ]:
# Analyze networks and identify significant interactions
significant_interactions = {}

for cell_line, network in network_results.items():
    print(f"\nAnalyzing network for {cell_line}...")
    
    # Analyze network
    interactions = analyze_networks(network, cell_line=cell_line)
    significant_interactions[cell_line] = interactions
    
    print(f"  Significant interactions: {len(interactions)}")
    if len(interactions) > 0:
        print(f"  Top interactions:")
        top_interactions = interactions.head()
        for _, row in top_interactions.iterrows():
            print(f"    {row['TF']} -> {row['target']} (importance: {row['importance']:.3f})")

# Combine results across cell lines
print(f"\n{'='*50}")
print("Summary across all cell lines:")
print(f"{'='*50}")

total_interactions = 0
for cell_line, interactions in significant_interactions.items():
    print(f"{cell_line}: {len(interactions)} interactions")
    total_interactions += len(interactions)

print(f"\nTotal significant interactions: {total_interactions}")

## 3.4 Save Results

Save the network analysis results for use in Stage 4.

In [ ]:
# Create output directory
output_dir = "networks"
os.makedirs(output_dir, exist_ok=True)

# Save individual cell line results
for cell_line, interactions in significant_interactions.items():
    output_file = f"{output_dir}/morphogen_regulon_network_{cell_line}.csv"
    interactions.to_csv(output_file, index=False)
    print(f"Saved {cell_line} network: {output_file}")

# Save combined results
all_interactions = []
for cell_line, interactions in significant_interactions.items():
    interactions_copy = interactions.copy()
    interactions_copy['cell_line'] = cell_line
    all_interactions.append(interactions_copy)

if all_interactions:
    combined_interactions = pd.concat(all_interactions, ignore_index=True)
    combined_file = f"{output_dir}/morphogen_regulon_networks_combined.csv"
    combined_interactions.to_csv(combined_file, index=False)
    print(f"Saved combined networks: {combined_file}")

print("\n✅ Stage 3 complete!")
print("Next step: Stage 4 - Final correlation analysis")